## Section 1: Data Cleaning

In [ ]:
import os
import pandas as pd
from geopy.distance import geodesic
from datetime import datetime
import re
import json
import glob

In [ ]:
os.mkdir('Data')

In [10]:
input_dir = 'careems_data/'
output_dir = 'clean_data/json_traj/'

# Get a list of all input files in the directory
input_files = [f for f in os.listdir(input_dir) if f.startswith('pooling_pings_') and f.endswith('.csv')]

#extract the date from filename
def extract_date_from_filename(filename):
    #regular expression to extract the date in the format YYYY-MM-DD
    match = re.search(r"\d{4}-\d{2}-\d{2}", filename)
    
    if match:
        return match.group(0)  #return extracted date
    else:
        raise ValueError("Date not found in filename. Expected format: trajectories-YYYY-MM-DD.csv")

#get day of the week from date string
def day_of_week(date_str):

    date = datetime.strptime(date_str, "%Y-%m-%d")

    #get day of the week (Monday is 0, Sunday is 6)
    day_index = date.weekday()

    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

    #get day of the week from the index
    day_name = days[day_index]

     #get day of the month (from 0 to 30)
    day_of_month = date.day - 1  

    return day_index, day_name, day_of_month

#get time ID (minute of the day from 0 to 1439)
def time_id_from_timestamp(timestamp_str):
    time = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
    total_minutes = time.hour * 60 + time.minute

    return total_minutes

def process_file(input_file, output_file):
    df = pd.read_csv(input_file)

    #convert to datetime
    df['location_read_at'] = pd.to_datetime(df['location_read_at'])

    #calculate distance between two points
    def calculate_distance(lat1, lon1, lat2, lon2):
        return geodesic((lat1, lon1), (lat2, lon2)).kilometers

    #calculate time difference in seconds
    def calculate_time_difference(time1, time2):
        return (time2 - time1).total_seconds()

    trip_data = []

    null_booking_id = '9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0'
    filtered_df = df[df['hash_booking_id'] != null_booking_id]  #filter out the null booking id
    grouped = filtered_df.groupby('hash_booking_id')

    for booking_id, group in grouped:
        #sort pings by timestamp
        group = group.sort_values(by='location_read_at')

        driver_id = group['hash_driver_id'].iloc[-1]

        #first instance of driver id to track switches
        first_instance = group[group['hash_driver_id'] == driver_id].iloc[0]

        time_id = first_instance['location_read_at']

        #filter out pings before switch
        valid_group = group[group['location_read_at'] >= time_id]

        lngs = valid_group['longitude'].tolist()
        lats = valid_group['latitude'].tolist()

        #dist gaps
        dist_gaps = [0]
        time_gaps = [0]
        prev_lat = lats[0]
        prev_lng = lngs[0]
        prev_time = time_id
        cum_dist = 0

        #total distance
        for lat, lng in zip(lats[1:], lngs[1:]):
            dist = calculate_distance(prev_lat, prev_lng, lat, lng)
            cum_dist += dist
            dist_gaps.append(cum_dist)
            prev_lat = lat
            prev_lng = lng

        total_dist = cum_dist

        #time gaps
        time_gaps = [(t - time_id).total_seconds() for t in valid_group['location_read_at']]

        #total time
        total_time = calculate_time_difference(valid_group['location_read_at'].iloc[0], valid_group['location_read_at'].iloc[-1])

        trip_data.append([booking_id, driver_id, time_id, lngs, lats, total_dist, total_time, time_gaps, dist_gaps])

    output_df = pd.DataFrame(trip_data, columns=['booking_id', 'driver_id', 'time_id', 'lngs', 'lats', 'dist', 'time', 'time_gap', 'dist_gap'])

    # Convert DataFrame to list of dicts for JSON processing
    trip_data_dicts = output_df.to_dict('records')

    # Create JSON objects
    json_data = []
    date_str = extract_date_from_filename(input_file)

    for row in trip_data_dicts:
        # Day of the week
        week_id, name, date_id = day_of_week(date_str)
        # Time ID is minute of day
        time_id = time_id_from_timestamp(str(row['time_id']))

        new_dict = {
            'trip_id': row['booking_id'],
            'time_gap': row['time_gap'],
            'dist': float(row['dist']),
            'lats': row['lats'],
            'driverID': row['driver_id'],
            'weekID': week_id,
            'timeID': time_id,
            'dateID': date_id,
            'time': float(row['time']),
            'lngs': row['lngs'],
            'dist_gap': row['dist_gap']
        }
        json_str = json.dumps(new_dict, separators=(',', ':'))  # Convert to JSON string
        json_data.append(json_str)

    # Write JSON data to file
    with open(output_file, 'w', encoding='utf-8') as output_file:
        for entry in json_data:
            output_file.write(entry + '\n')
            output_file.flush()

# Process each input file
for input_file in input_files:
    date_str = input_file.split('_')[-1].split('.')[0]  # Extract date from file name
    output_file = os.path.join(output_dir, f'traj_fix_dist_{date_str}.json')
    process_file(os.path.join(input_dir, input_file), output_file)
    print(f"Data has been written to {output_file}")

Data has been written to json_traj/traj_fix_dist_2024-01-28.json


### Extract clean trips

In [6]:
input_dir = 'clean_data/json_traj/'
output_dir = 'clean_data/'
clean_file_template = "clean_{date}.json"

# Load pooling data
poolin_dir = 'careems_data/'
dp = []

# Loop through all pooling files 
for file_name in os.listdir(poolin_dir):
    if file_name.startswith('anon_pooling') and file_name.endswith('.csv'):  # Check file name and extension
        file_path = os.path.join(poolin_dir, file_name)
        df = pd.read_csv(file_path)  
        dp.append(df)  
pooling_data = pd.concat(dp, ignore_index=True)


for j in os.listdir(input_dir):
    date_str = j.split('_')[1].split('.')[0]  # Extract date from file name
    filtered_pooling_data = pooling_data[pooling_data['day'] == date_str]
    
    if filtered_pooling_data.empty:
        print(f"No pooling data for date: {date_str}")
        continue

    input_file_path = os.path.join(input_dir, j)
    clean_file_path = os.path.join(output_dir, clean_file_template.format(date=date_str))

    with open(input_file_path, "r") as json_traj_file:
        new_data = [json.loads(line) for line in json_traj_file]  # Read each line as a JSON object

    with open(clean_file_path, 'w', encoding='utf-8') as clean_file:
        for entry in new_data:
            entry_time = entry["time"]
            entry_trip_id = entry["trip_id"]
            entry_driver_id = entry["driverID"]

            for _, row in filtered_pooling_data.iterrows():
                pool_time = float(row["captain_engagement_time"] * 60)
                pool_trip_id = row["booking_id"]
                pool_driver_id = row["captain_id"]
                time_diff = pool_time - entry_time

                # Adding time difference to entry
                entry_with_time_diff = entry.copy()
                entry_with_time_diff["time_diff"] = time_diff

                # Writing good trips to clean file (same trip and driver id and <= 5 sec time diff)
                if entry_trip_id == pool_trip_id and entry_driver_id == pool_driver_id and abs(time_diff) <= 5:
                    json.dump(entry_with_time_diff, clean_file)
                    clean_file.write("\n")
                    break

    print(f"Processed and cleaned data for date: {date_str}")

print("Matching process completed. Check the 'trial week/clean_data/' directory for results.")


Processed and cleaned data for date: 2024-01-22
Processed and cleaned data for date: 2024-01-11
Processed and cleaned data for date: 2024-01-23
Processed and cleaned data for date: 2024-01-01
Processed and cleaned data for date: 2024-01-21
Processed and cleaned data for date: 2024-01-06
Processed and cleaned data for date: 2024-01-12
Processed and cleaned data for date: 2024-01-05
Processed and cleaned data for date: 2024-01-17
Processed and cleaned data for date: 2024-01-24
Processed and cleaned data for date: 2024-01-09
Processed and cleaned data for date: 2024-01-15
Processed and cleaned data for date: 2024-01-13
Processed and cleaned data for date: 2024-01-04
Processed and cleaned data for date: 2024-01-14
Processed and cleaned data for date: 2024-01-16
Processed and cleaned data for date: 2024-01-03
Processed and cleaned data for date: 2024-01-18
Processed and cleaned data for date: 2024-01-08
Processed and cleaned data for date: 2024-01-19
Processed and cleaned data for date: 202

## Section 2: Splitting Trips

### Step 1: Import Libraries

In [1]:
import pandas as pd
import json
import numpy as np
import csv
from datetime import datetime
import os
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from sklearn.cluster import KMeans
import numpy as np
from shapely.geometry import MultiPoint
from shapely.ops import unary_union
import matplotlib.pyplot as plt

In [3]:
os.chdir('Data')

### Step 2: Load Data

In [5]:
# Load merchant data
merchant_file_path = 'careems_data/order_merchant_id_anon.parquet'
merchants_df = pd.read_parquet(merchant_file_path)

In [4]:
# Load pooling data
directory = 'careems_data/'
dp = []

# Loop through all pooling files 
for file_name in os.listdir(directory):
    if file_name.startswith('anon_pooling') and file_name.endswith('.csv'):  # Check file name and extension
        file_path = os.path.join(directory, file_name)
        df = pd.read_csv(file_path)  
        dp.append(df)  


df_pooling = pd.concat(dp, ignore_index=True)

In [7]:
# Load clean data and drop time_diff column
directory = 'clean_data/'
dp = []

# Loop through all clean data files 
for file_name in os.listdir(directory):
    if file_name.startswith('clean_2024-') and file_name.endswith('.json'):  # Check file name and extension
        month = int(file_name.split('-')[1])  # Extract the month and convert to int
        file_path = os.path.join(directory, file_name)  
        with open(file_path, 'r') as file:
            for line in file:  
                entry = json.loads(line)  
                entry['month'] = month  # Add month to the data
                dp.append(entry) 


df = pd.DataFrame(dp)

if 'time_diff' in df.columns:
    df.drop('time_diff', axis=1, inplace=True)

df = df.rename(columns={'time': 'trip_time'})

df.dropna(inplace=True)

#### Left join merchants on the pings df

In [8]:
# Merge merchants_df with df_pooling on 'order_id'
merged_pooling = df_pooling.merge(merchants_df[['order_id', 'merchant_id']], on='order_id', how='left')

# Merge with df on 'trip_id' 
df = df.merge(merged_pooling[['booking_id', 'merchant_id']], left_on='trip_id', right_on='booking_id', how='left')

df = df.drop(columns=['booking_id'])

# Check how many 'merchant_id' values are null 
print(df['merchant_id'].isnull().sum())

0


### Step 3: Stationarity Detection

#### Find all stationary intervals over each trip

In [11]:
def detect_stationary_intervals(df):
    results = []
    for index, row in df.iterrows():
        trip_id = row['trip_id']
        lats = np.array(row['lats'])
        lngs = np.array(row['lngs'])
        time_gap = np.array(row['time_gap'])
        
        # Compute differences between consecutive points directly from lat/lng arrays
        lat_diff = np.abs(np.diff(lats))
        lng_diff = np.abs(np.diff(lngs))

        # Identify stationary intervals
        stationary_indices = np.where((lat_diff == 0) & (lng_diff == 0))[0]
        
        # Group consecutive stationary indices into intervals
        if len(stationary_indices) > 0:
            start_idx = stationary_indices[0]
            for i in range(1, len(stationary_indices)):
                # If the current index is not consecutive, close the interval
                if stationary_indices[i] != stationary_indices[i - 1] + 1:
                    end_idx = stationary_indices[i - 1]
                    # Save the interval only if it has more than one index
                    if end_idx > start_idx:
                        interval = {
                            "trip_id": trip_id,
                            "start_idx": int(start_idx),
                            "end_idx": int(end_idx)+1,
                            "start_lat": float(lats[start_idx]),
                            "start_lng": float(lngs[start_idx]),
                            "end_lat": float(lats[end_idx+1]),
                            "end_lng": float(lngs[end_idx+1]),
                            "time_elapsed": float(time_gap[end_idx+1] - time_gap[start_idx])
                        }
                        results.append(interval)
                    # Start a new interval
                    start_idx = stationary_indices[i]
            # Add the last interval if it has more than one index
            end_idx = stationary_indices[-1]
            if end_idx > start_idx:
                interval = {
                    "trip_id": trip_id,
                    "start_idx": int(start_idx),
                    "end_idx": int(end_idx)+1,
                    "start_lat": float(lats[start_idx]),
                    "start_lng": float(lngs[start_idx]),
                    "end_lat": float(lats[end_idx+1]),
                    "end_lng": float(lngs[end_idx+1]),
                    "time_elapsed": float(time_gap[end_idx+1] - time_gap[start_idx])
                }
                results.append(interval)

    results_df = pd.DataFrame(results)
    return results_df

stationary_df_pre_truncation = detect_stationary_intervals(df)

#### Truncate trips where a stationary interval exists at the end of trip

In [12]:

def truncate_trips_if_stationary(df, stationary_df):
    truncated_trips = []

    # Iterate over each trip in the df
    for index, row in df.iterrows():
        trip_id = row['trip_id']
        lats = row['lats']
        lngs = row['lngs']
        time_gap = row['time_gap']
        dist_gap = row['dist_gap']
        dist = row['dist']
        trip_time = row['trip_time']
        month = row['month']

        # Get the last coordinates of the trip
        last_lat, last_lng = lats[-1], lngs[-1]

        # Find the corresponding stationary intervals for this trip
        intervals = stationary_df[stationary_df['trip_id'] == trip_id]

        if not intervals.empty:
            # Get the last stationary interval for this trip
            last_interval = intervals.iloc[-1]

            interval_lat, interval_lng = last_interval['end_lat'], last_interval['end_lng']
            time_elapsed = last_interval['time_elapsed']

            # Check if the last stationary interval's coordinates match the last trip coordinates
            if (last_lat == interval_lat and last_lng == interval_lng) and time_elapsed > 10:
                # Keep only the first part of the stationary segment
                truncated_trip = {
                    'trip_id': trip_id,
                    'lats': lats[:last_interval['start_idx'] + 1],
                    'lngs': lngs[:last_interval['start_idx'] + 1],
                    'time_gap': time_gap[:last_interval['start_idx'] + 1],
                    'dist_gap': dist_gap[:last_interval['start_idx'] + 1],
                    'dist': dist,  # Keep the original distance
                    'trip_time': time_gap[last_interval['start_idx']],  # Adjusted trip time
                    'driverID': row['driverID'],
                    'weekID': row['weekID'],
                    'timeID': row['timeID'],
                    'dateID': row['dateID'],
                    'merchant_id': row['merchant_id'],
                    'month': month
                }
                truncated_trips.append(truncated_trip)
            else:
                # If no truncation is needed, keep the original trip
                truncated_trips.append(row.to_dict())
        else:
            # If no stationary intervals exist, keep the original trip
            truncated_trips.append(row.to_dict())

    # Convert the list of truncated trips back into a df
    truncated_df = pd.DataFrame(truncated_trips)

    # Ensure all columns match the original schema
    for column in df.columns:
        if column not in truncated_df.columns:
            truncated_df[column] = None  # Add missing columns with default None values

    # Reorder columns to match the original DataFrame
    truncated_df = truncated_df[df.columns]

    return truncated_df

truncated_df = truncate_trips_if_stationary(df, stationary_df_pre_truncation)


#### Find new stationary intervals post truncation and longest interval for each trip

In [13]:
def detect_stationary_intervals(df):
    results = []
    for index, row in df.iterrows():
        trip_id = row['trip_id']
        lats = np.array(row['lats'])
        lngs = np.array(row['lngs'])
        time_gap = np.array(row['time_gap'])
        
        # Compute differences between consecutive points directly from lat/lng arrays
        lat_diff = np.abs(np.diff(lats))
        lng_diff = np.abs(np.diff(lngs))

        # Identify stationary intervals 
        stationary_indices = np.where((lat_diff == 0) & (lng_diff == 0))[0]
        
        # Group consecutive stationary indices into intervals
        if len(stationary_indices) > 0:
            start_idx = stationary_indices[0]
            for i in range(1, len(stationary_indices)):
                # If the current index is not consecutive, close the interval
                if stationary_indices[i] != stationary_indices[i - 1] + 1:
                    end_idx = stationary_indices[i - 1]
                    # Save the interval only if it has more than one index
                    if end_idx > start_idx:
                        interval = {
                            "trip_id": trip_id,
                            "start_idx": int(start_idx),
                            "end_idx": int(end_idx)+1,
                            "start_lat": float(lats[start_idx]),
                            "start_lng": float(lngs[start_idx]),
                            "end_lat": float(lats[end_idx+1]),
                            "end_lng": float(lngs[end_idx+1]),
                            "time_elapsed": float(time_gap[end_idx+1] - time_gap[start_idx])
                        }
                        results.append(interval)
                    # Start a new interval
                    start_idx = stationary_indices[i]
            # Add the last interval if it has more than one index
            end_idx = stationary_indices[-1]
            if end_idx > start_idx:
                interval = {
                    "trip_id": trip_id,
                    "start_idx": int(start_idx),
                    "end_idx": int(end_idx)+1,
                    "start_lat": float(lats[start_idx]),
                    "start_lng": float(lngs[start_idx]),
                    "end_lat": float(lats[end_idx+1]),
                    "end_lng": float(lngs[end_idx+1]),
                    "time_elapsed": float(time_gap[end_idx+1] - time_gap[start_idx])
                }
                results.append(interval)

    results_df = pd.DataFrame(results)
    return results_df

stationary_df = detect_stationary_intervals(truncated_df)

In [14]:
df=truncated_df

In [9]:
def extract_longest_stationary_interval(stationary_df):
    # Find the longest stationary interval for each trip_id based on time_elapsed
    longest_intervals = stationary_df.loc[stationary_df.groupby('trip_id')['time_elapsed'].idxmax()]

    longest_intervals.reset_index(drop=True, inplace=True)
    
    return longest_intervals

longest_intervals_df = extract_longest_stationary_interval(stationary_df)

##### Combined df with pooling coordinates and longest stationary interval coordinates

In [20]:
def create_combined(longest_intervals_df, pooling_df):
    
    # Rename 'booking_id' to 'trip_id' in pooling_df for consistency
    pooling_df = pooling_df.rename(columns={'booking_id': 'trip_id'})
    
    combined_df = (
        longest_intervals_df[['trip_id', 'start_lat', 'start_lng']]
        .merge(pooling_df[['trip_id', 'pickup_latitude', 'pickup_longitude']], on='trip_id', how='inner')
    )

    # Rename columns for clarity in the combined data
    combined_df = combined_df.rename(columns={
        'start_lat': 'stationary_df_lat',
        'start_lng': 'stationary_df_lng',
        'pickup_latitude': 'pooling_df_lat',
        'pickup_longitude': 'pooling_df_lng'
    })   
    return combined_df

combined_df = create_combined(longest_intervals_df, df_pooling)

combined_df = combined_df.merge(df[['trip_id', 'merchant_id']], on='trip_id', how='left')

#### Update incorrect pooling coordinates

In [21]:
def check_inconsistent_groups_with_tolerance_fix(combined_df):
    
    # Lists to store inconsistent groups and detailed non-unique counts
    inconsistent_groups = []
    nonunique_info = []

    for merchant_id, group in combined_df.groupby('merchant_id'):
        # Extract latitude and longitude values for the group
        lat_values = group['pooling_df_lat'].values
        lng_values = group['pooling_df_lng'].values

        # Count occurrences of each latitude and longitude
        lat_counts = pd.Series(lat_values).value_counts()
        lng_counts = pd.Series(lng_values).value_counts()

        # Identify the majority (most common) latitude and longitude
        majority_lat = lat_counts.idxmax()
        majority_lng = lng_counts.idxmax()

        # Filter out the non-majority coordinates
        non_majority_lats = lat_counts[lat_counts.index != majority_lat]
        non_majority_lngs = lng_counts[lng_counts.index != majority_lng]

        # If there are non-majority coordinates, store the information
        if not non_majority_lats.empty or not non_majority_lngs.empty:
            inconsistent_groups.append(group)

            # Store the non-unique information for this merchant_id
            nonunique_info.append({
                'merchant_id': merchant_id,
                'non_majority_lats': non_majority_lats.to_dict(),  # Non-majority latitudes and their counts
                'non_majority_lngs': non_majority_lngs.to_dict(),  # Non-majority longitudes and their counts
                'group_size': len(group)
            })

    # Combine all inconsistent groups into a single DataFrame
    if inconsistent_groups:
        print(f"Number of inconsistent groups: {len(inconsistent_groups)}")
        non_majority_df = pd.DataFrame(nonunique_info)
    else:
        print("No inconsistencies found.")

    return non_majority_df

non_majority_df = check_inconsistent_groups_with_tolerance_fix(combined_df)


Number of inconsistent groups: 1775


In [24]:
# Change incorrect pooling coordinates in combined_df to the majority coordinates for each merchant
def correct_pooling_coordinates_with_majority(non_majority_df, combined_df):

    # Iterate over each merchant in the non-majority df
    for _, row in non_majority_df.iterrows():
        merchant_id = row['merchant_id']

        # Get the majority latitude and longitude for this merchant from combined_df
        majority_lat = combined_df[combined_df['merchant_id'] == merchant_id]['pooling_df_lat'].mode()[0]
        majority_lng = combined_df[combined_df['merchant_id'] == merchant_id]['pooling_df_lng'].mode()[0]

        # Find the trip_ids with non-majority coordinates
        merchant_group = combined_df[combined_df['merchant_id'] == merchant_id]
        incorrect_trips = merchant_group[
                (merchant_group['pooling_df_lat'] != majority_lat) |
                (merchant_group['pooling_df_lng'] != majority_lng)
            ]

        # Update the coordinates in combined_df for the incorrect trips
        for trip_id in incorrect_trips['trip_id']:
            combined_df.loc[combined_df['trip_id'] == trip_id, ['pooling_df_lat', 'pooling_df_lng']] = [
                majority_lat, majority_lng
            ]

    return combined_df


combined_df = correct_pooling_coordinates_with_majority(non_majority_df, combined_df)


#### Extract non-matching coordinates and drop them from df and longest stationary df

In [25]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    # Compute differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Apply Haversine formula
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c

    return distance

def extract_non_matching_coordinates(combined_df):
    # Identify rows where the stationary coordinates do not match pooling coordinates
    non_matching_df = combined_df[
        (combined_df['stationary_df_lat'] != combined_df['pooling_df_lat']) |
        (combined_df['stationary_df_lng'] != combined_df['pooling_df_lng'])
    ]

    # calculate the distance between the coordinates
    non_matching_df['distance_km'] = haversine(
        non_matching_df['stationary_df_lat'],
        non_matching_df['stationary_df_lng'],
        non_matching_df['pooling_df_lat'],
        non_matching_df['pooling_df_lng']
    )

    return non_matching_df

non_matching_entries = extract_non_matching_coordinates(combined_df)


/tmp/ipykernel_8074/54936362.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matching_df['distance_km'] = haversine(


In [26]:
# Drop non-matching trips from `df` and `longest_intervals_df`
def drop_non_matching_trips(df, longest_intervals_df, non_matching_entries):
    # Extract the trip IDs from non-matching entries
    non_matching_trip_ids = non_matching_entries['trip_id'].unique()

    # Drop these trips from both dfs
    df_filtered = df[~df['trip_id'].isin(non_matching_trip_ids)]
    longest_intervals_filtered = longest_intervals_df[~longest_intervals_df['trip_id'].isin(non_matching_trip_ids)]

    return df_filtered, longest_intervals_filtered


df, longest_intervals_df = drop_non_matching_trips(df, longest_intervals_df, non_matching_entries)
print("Filtered DataFrames saved.")

Filtered DataFrames saved.


### Step 4: Splitting trips into 3 segments (Driver-to-Merchant, Wait-Time-at-Merchant, Merchant-to-Customer)

#### Split trips based on stationary intervals matching coordinates from pooling

In [50]:

def adjust_gaps_with_original_pattern(original_gaps, new_gaps):
    """
    Adjust new gaps based on the pattern of change in the original gaps.
    The new gaps will start from 0 but follow the same incremental differences.
    """
    if len(original_gaps) <= 1 or len(new_gaps) == 0:
        return new_gaps  # No meaningful adjustment needed

    # Calculate the incremental changes (differences) in the original gaps
    original_diffs = np.diff(original_gaps)

    # Start the new gaps from 0
    adjusted_gaps = [0]

    # Apply the original differences to the new gaps
    for i in range(1, len(new_gaps)):
        diff = original_diffs[(i - 1) % len(original_diffs)]
        adjusted_gaps.append(adjusted_gaps[-1] + diff)

    return adjusted_gaps

def segment_trips(df, longest_intervals_df):
    road_list = []
    second_segment_list = []

    # Iterate over each entry in the df dataframe
    for index, row in df.iterrows():
        trip_id = row['trip_id']
        interval = longest_intervals_df[longest_intervals_df['trip_id'] == trip_id]
        lats = row['lats']
        lngs = row['lngs']
        time_gap = row['time_gap']
        dist_gap = row['dist_gap']
        month = row['month']

        if not interval.empty:
            start_idx = interval['start_idx'].values[0]
            end_idx = interval['end_idx'].values[0]

            # First segment
            first_segment = {
                'trip_id': trip_id,
                'time_gap': time_gap[:start_idx],
                'dist': dist_gap[start_idx - 1] if start_idx > 0 else 0,
                'trip_time': time_gap[start_idx - 1] if start_idx > 0 else 0,
                'driverID': row['driverID'],
                'weekID': row['weekID'],
                'timeID': row['timeID'],
                'dateID': row['dateID'],
                'dist_gap': dist_gap[:start_idx],
                'lats': lats[:start_idx],
                'lngs': lngs[:start_idx],
                'month': month,
                'time_offset': 0,
                'segmentID': 1
            }

            # Second segment (stationary)
            stationary_time_gap = time_gap[start_idx:end_idx + 1]
            stationary_dist_gap = dist_gap[start_idx:end_idx + 1]

            stationary_time_gap = [time - stationary_time_gap[0] for time in stationary_time_gap]
            stationary_dist_gap = [dist - stationary_dist_gap[0] for dist in stationary_dist_gap]

            second_segment = {
                'trip_id': trip_id,
                'time_gap': stationary_time_gap,
                'dist': stationary_dist_gap[-1] if len(stationary_dist_gap) > 0 else 0,
                'trip_time': stationary_time_gap[-1] if len(stationary_time_gap) > 0 else 0,
                'driverID': row['driverID'],
                'weekID': row['weekID'],
                'timeID': row['timeID'],
                'dateID': row['dateID'],
                'dist_gap': stationary_dist_gap,
                'lats': lats[start_idx:end_idx + 1],
                'lngs': lngs[start_idx:end_idx + 1],
                'month': month,
                'time_offset': time_gap[start_idx],
                'merchant': row['merchant_id']
            }

            # Third segment (after stationary)
            new_time_gap = time_gap[end_idx + 1:]
            new_dist_gap = dist_gap[end_idx + 1:]

            # Adjust the gaps using the pattern from the original trip
            adjusted_time_gap = adjust_gaps_with_original_pattern(time_gap[end_idx + 1:], new_time_gap)
            adjusted_dist_gap = adjust_gaps_with_original_pattern(dist_gap[end_idx + 1:], new_dist_gap)
            time_offset3 = time_gap[end_idx + 1] if len(time_gap) > end_idx + 1 else 0

            third_segment = {
                'trip_id': trip_id,
                'time_gap': adjusted_time_gap,
                'dist': adjusted_dist_gap[-1] if len(adjusted_dist_gap) > 0 else 0,
                'trip_time': adjusted_time_gap[-1] if len(adjusted_time_gap) > 0 else 0,
                'driverID': row['driverID'],
                'weekID': row['weekID'],
                'timeID': row['timeID']+round((time_offset3/60),1),
                'dateID': row['dateID'],
                'dist_gap': adjusted_dist_gap,
                'lats': lats[end_idx + 1:],
                'lngs': lngs[end_idx + 1:],
                'month': month,
                'time_offset': time_offset3,
                'segmentID': 3
            }
        else:
            print(f"No interval found for trip_id: {trip_id}")

        # Add segments to their respective lists
        if len(first_segment['time_gap']) > 1:
            road_list.append(first_segment)

        if len(third_segment['time_gap']) > 1:
            road_list.append(third_segment)

        if len(second_segment['time_gap']) > 1:
            second_segment_list.append(second_segment)

    # Convert the lists to DataFrames
    road_df = pd.DataFrame(road_list)
    second_segment_df = pd.DataFrame(second_segment_list)

    return road_df, second_segment_df

road_df, second_segment_df = segment_trips(df, longest_intervals_df)

No interval found for trip_id: 65d99e964c50a5eed233fe9a7864114c1d91cf6b6ba542bbf4906a998d4df27f
No interval found for trip_id: 7d71d55894365ccad46d7e7c533997c58e200cc235919b0c817944af8b5226a1
No interval found for trip_id: 218225707aa59531afe509f5ff2b52200dbdd449550265a226f348a814421952
No interval found for trip_id: bec1e8f0b569297918a23501ea481f66e0d3417dd07f96b91b7db23d257bd8d1
No interval found for trip_id: cc31cd6a0490a3f2ba6811353e2f2339817b2b7a3088e4ad26dae93ad6159f9e
No interval found for trip_id: 2d3579c7f52c46c1bb872ff4c4a7ffe5c1c44cebf887bb56081c8f52f2039949
No interval found for trip_id: 71e5adb6c386c8735b7d0c84db5bcd80390d36d3e35a459dd544df0405deb66e
No interval found for trip_id: 764938195bf3e3ff342db9708afc6a92b72ce7c8b41137a15bca38a910c4722e
No interval found for trip_id: 025c623bc8fc62c7b3f4618767d39078a94b96156bf307c010a33ab270988e26
No interval found for trip_id: aebd4aab90fe81bc19cb4c0be8a13250b63201a9059205e56a21caa496920125
No interval found for trip_id: d415bf3c4

In [51]:
# Remove trips with single 3rd segment occurences
single_occurrence_trips = road_df['trip_id'].value_counts()
single_occurrence_trips = single_occurrence_trips[single_occurrence_trips == 1].index

filtered_single_trips_df = road_df[road_df['trip_id'].isin(single_occurrence_trips)]

filtered_single_trips_segment3_df = filtered_single_trips_df[filtered_single_trips_df['segmentID'] == 1]

# Remove these trips from road_df
road_df = road_df[~road_df['trip_id'].isin(filtered_single_trips_segment3_df['trip_id'])]

# Extract unique combinations of dateID and month from road_df
unique_date_month_combinations = road_df[['dateID', 'month']].drop_duplicates()

# Iterate over each unique combination
for _, row in unique_date_month_combinations.iterrows():
    date_id = row['dateID']
    month = row['month']
    
    # Filter road_df and second_segment_df for the current dateID and month
    road_df_filtered = road_df[(road_df['dateID'] == date_id) & (road_df['month'] == month)]
    second_segment_df_filtered = second_segment_df[(second_segment_df['dateID'] == date_id) & (second_segment_df['month'] == month)]
    
    # Convert to JSON format
    road_json = road_df_filtered.to_dict(orient='records')
    second_seg_json = second_segment_df_filtered.to_dict(orient='records')

    # Generate filenames with month and dateID + 1
    road_file_name = f'segmented_trips/Segmented_Trips_0{month}_{date_id + 1}.json'
    segment_file_name = f'segmented_trips/Merchants_Segments_0{month}_{date_id + 1}.json'

    # Save files
    with open(road_file_name, 'w') as file:
        for json_obj in road_json:
            json.dump(json_obj, file)
            file.write('\n')

    with open(segment_file_name, 'w') as file:
        for json_obj in second_seg_json:
            json.dump(json_obj, file)
            file.write('\n')

    print(f"Segments 1 & 3 saved to '{road_file_name}'")
    print(f"Segment 2 with wait times saved to '{segment_file_name}'")

Segments 1 & 3 saved to 'segmented_trips/Segmented_Trips_01_27.json'
Segment 2 with wait times saved to 'segmented_trips/Merchants_Segments_01_27.json'
Segments 1 & 3 saved to 'segmented_trips/Segmented_Trips_02_6.json'
Segment 2 with wait times saved to 'segmented_trips/Merchants_Segments_02_6.json'
Segments 1 & 3 saved to 'segmented_trips/Segmented_Trips_02_17.json'
Segment 2 with wait times saved to 'segmented_trips/Merchants_Segments_02_17.json'
Segments 1 & 3 saved to 'segmented_trips/Segmented_Trips_02_16.json'
Segment 2 with wait times saved to 'segmented_trips/Merchants_Segments_02_16.json'
Segments 1 & 3 saved to 'segmented_trips/Segmented_Trips_02_19.json'
Segment 2 with wait times saved to 'segmented_trips/Merchants_Segments_02_19.json'
Segments 1 & 3 saved to 'segmented_trips/Segmented_Trips_02_7.json'
Segment 2 with wait times saved to 'segmented_trips/Merchants_Segments_02_7.json'
Segments 1 & 3 saved to 'segmented_trips/Segmented_Trips_01_5.json'
Segment 2 with wait time

In [52]:
road_df.describe()

,dist,trip_time,weekID,timeID,dateID,month,time_offset,segmentID
count,499600.000000,499600.000000,499600.000000,499600.000000,499600.000000,499600.000000,499600.000000,499600.000000
mean,4.317217,537.415412,2.871511,876.788028,14.962560,1.471910,418.560200,2.120997
std,32.353432,441.039350,1.944810,281.038293,8.837051,0.499211,491.149592,0.992654
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
25%,0.761955,195.000000,1.000000,672.400000,7.000000,1.000000,0.000000,1.000000
50%,2.331419,423.000000,3.000000,894.100000,15.000000,1.000000,304.000000,3.000000
75%,5.699662,775.000000,4.000000,1097.100000,23.000000,2.000000,711.000000,3.000000
max,6807.365422,9684.000000,6.000000,1440.100000,30.000000,2.000000,6442.000000,3.000000


In [53]:
second_segment_df.describe()

,dist,trip_time,weekID,timeID,dateID,month,time_offset
count,280669.0,280669.000000,280669.000000,280669.000000,280669.000000,280669.000000,280669.000000
mean,0.0,517.142053,2.866227,871.998511,14.907104,1.475756,216.236396
std,0.0,368.269721,1.949826,282.633940,8.800414,0.499413,238.066241
min,0.0,10.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.0,248.000000,1.000000,666.000000,7.000000,1.000000,40.000000
50%,0.0,416.000000,3.000000,891.000000,15.000000,1.000000,155.000000
75%,0.0,686.000000,5.000000,1094.000000,23.000000,2.000000,310.000000
max,0.0,5824.000000,6.000000,1437.000000,30.000000,2.000000,4519.000000
